In [ ]:
from examples.pH_calibration import pH_calibration as pH_calibration
import sbol3
from paml.execution_engine import ManualExecutionEngine
import paml
import uml
import tyto
from IPython.display import display, HTML
import types

%load_ext autoreload
%autoreload 2

In [ ]:
def setup_primitives(
    doc, 
    manual_primitives = [\
        'https://bioprotocols.org/paml/primitives/pH_calibration/PHMeterCalibrated',
        'https://bioprotocols.org/paml/primitives/pH_calibration/MeasurePH',
        'https://bioprotocols.org/paml/primitives/pH_calibration/MeasureTemperature',
        'https://bioprotocols.org/paml/primitives/pH_calibration/CalculateNaOHAddition',
        'https://bioprotocols.org/paml/primitives/pH_calibration/AtTargetPH'
    ]):
    """Delete any implementations of compute_output for the primitives listed in manual_primitives. Call initialize_primitive_compute_output to set compute_output for primitives with known implementations.

    Args:
        doc (sbol3.Document): Document that has the primitive objects.
    """
    primitives = [p for p in doc.objects if isinstance(p, paml.Primitive)]
    primitives_to_unset = [p for p in primitives if p.identity in manual_primitives]
    for x in primitives_to_unset:
        x.compute_output = types.MethodType(paml.primitive_execution.primitive_compute_output, x)
    
    paml.initialize_primitive_compute_output(doc)

In [ ]:
# Make the protocol
new_protocol, doc = pH_calibration.pH_calibration_protocol()
setup_primitives(doc)

# Setup ExecutionEngine
ee = ManualExecutionEngine()
ee.initialize(new_protocol, sbol3.Agent("test_agent"), id="test_execution", parameter_values=[
    paml.ParameterValue(parameter=new_protocol.get_input("reaction_volume"), value=sbol3.Measure(10, tyto.OM.milliliter)),
])

# Start execution of protocol, and stop at point needing manual input.
ready, choices, graph = ee.run(new_protocol)

# List the choices of next Activity to execute
print("********************** Possible Next Activities: **********************")
print(choices)
print("*"*80)
# Render the state of execution as a graph (Blue = Completed, Green = Possible Next (i.e., ready))
graph

In [ ]:
# Execute the next step that determines whether the pH meter is calibrated

def is_calibrated(node: paml.ActivityNodeExecution,
                  parameter: uml.Parameter):
    return False  # The pH meter is not calibrated!

# Execute the next step using is_calibrated to determine its output
ready, choices, graph = ee.next(ready[0], node_output=is_calibrated)
print(choices)
graph

In [ ]:
# Check whether calibration was successful

def is_calibration_successful(node: paml.ActivityNodeExecution,
                  parameter: uml.Parameter):
    return True  # It was successful!

ready, choices, graph = ee.next(ready[0], node_output=is_calibration_successful)
print(choices)
graph

In [ ]:
# Get the temperature measurement

def measure_temperature(node: paml.ActivityNodeExecution,
                  parameter: uml.Parameter):
    return sbol3.Measure(30, "http://www.ontology-of-units-of-measure.org/resource/om-2/degreeCelsius")

measure_temp = next((r for r in ready if r.behavior.lookup().display_id == "MeasureTemperature"))

ready, choices, graph = ee.next(measure_temp, node_output=measure_temperature)
print(choices)
graph

In [ ]:
# Get the pH measurement 

def measure_pH(node: paml.ActivityNodeExecution,
                  parameter: uml.Parameter):
    return uml.literal(7)

m_pH = next((r for r in ready if r.behavior.lookup().display_id == "MeasurePH"))

ready, choices, graph = ee.next(m_pH, node_output=measure_pH)
print(choices)
graph

In [ ]:
# Is the target pH met?
def at_target(node: paml.ActivityNodeExecution,
                  parameter: uml.Parameter):
    return uml.literal(eval(input())) # Use input "True" or "False"

at_t_pH = next((r for r in ready if r.behavior.lookup().display_id == "AtTargetPH"))

ready, choices, graph = ee.next(at_t_pH, node_output=at_target)
print(choices)
graph

In [ ]:
ee.finalize(new_protocol)
ee.ex.completed_normally